## Exercise 1 (30 points)

### Reinforcement Learning with Function Approximation:
- 1.1 (15 points): List some pros and cons of model-based vs. value-based vs. policy-based reinforcement learning. Provide at least 2 pros and 2 cons for each method.
- 1.2 (5 points): Why does parameterizing a function can help scale and generalize through larger state and/or action spaces?
- 1.3 (10 points): Discuss the trade-offs between using simulated experiences and real experiences in reinforcement learning. Can a combination of both be beneficial? Support your answer with a scenario where either approach alone is preferable.

<span style="color: lightgreen;" >

- 1.1 
    - For model-based RL like MDP with DP:
        - The pros would be:
            - It can simulate experiences from the learned transitional model($P(s'|s, a)$) and need fewer real interactions with the environment. 
            - We can use the model to look ahead via methods like Monte Carlo Tree Search to look ahead and improve decision-making.
        - The cons would be:
            - The learned model most of time could be biased and lead to poor decisions.
            - Deriving a policy from planning is non-trivial since even with a known model, we must still sole the MDP to get the optimal policy.
            - May focus compute/capacity on irrelevant details
    - For value-based method:
        - The pros would be:
            - It is efficient in discrete action spaces. When the action space is small(like in some grid-world example), it's easy to enumerate all possible actions and choose the best one.
            - Value-based methods like Q-learning are off-policy by nature, which allows learning from past episodes/samples(e.g. replay buffer in DQN).
        - The cons would be:
            - Not suitable for continuous action spaces since in the algorithm we need to compute $\max_{a}Q(s, a)$ where this maximum operator is generally defined on discrete function.
            - The policy learned from values is not explicitly optimized. In complex environments, good exploration is needed. Otherwise, the algorithm would result in suboptimal behavior.
    - For policy-based methods:
        - The pros would be:
            - They work well with continuous and high dimension action spaces.
            - They train the policy to maximize expected reward directly and thus there is no need to learn value functions first.
        - The cons would be:
            - Policy gradient estimates are often noisy and would typically require variance reduction tricks like baselines or GAE method.
            - They can collapse to suboptimal deterministic policies early in training (premature convergence).
            - Ignores all other learnable knowledge(e.g. $V(s)$ and $Q(s, a)$) so may not efficiently use the available data
- 1.2 There are two reasons for this: 
    - In large or continuous spaces, it's impossible to store a value or policy for every single state or action explicitly, which would require large amount of memory. 
    - A non-parametric table-based method only knows what it's seen before. However, a parameterized function (like a neural net or linear function) can interpolate or extrapolate to new states it hasn't explicitly encountered.
- 1.3 The reason why combining may help is that simulated experiences can improve samlpe efficiency while real experiences can help correct for model errors and keep the agent grounded in reality. For these high-risk, real-world systems, like autonomous driving, real experiences would be better. For these scenarios like if you want to train an AI to play a video game, the simulation is preferred since it's fast, not memory/computationally-intensive and perfectly accurate.
</span>

##  Exercise 2 (30 points)

### Actor-Critic Reinforcement Learning::
- 2.1 (10 points): Provide at least 3 examples of objective measure that have been successfully used (i.e., published or discussed in the course) for policy optimization using gradient ascent.
- 2.2 (5 points): In Actor-Critic RL, why does approximating values to define the Critic can help increase the Actor’s sampling efficiency?
- 2.3 (5 points): What is the impact of the regularization term added to the objective measure in the TRPO algorithm?
- 2.4 (5 points) : Compare PPO and TRPO. Why PPO is generally preferred in practical implementations. Provide an example of a reinforcement learning scenario where PPO’s advantages are particularly beneficial.
- 2.5 (5 points) : Write down the policy gradient theorem. Explain how sampling contributes to estimating the policy gradient and why it is necessary for effective learning in reinforcement learning.

<span style="color: lightgreen;" >

- 2.1 The measures are as follows:
    - **Episodic expected return objective:** $J_{0}(\theta)=\mathbb{E}_{\pi_{\theta}}(G_{t}|s_{0})$ This is the standard reinforce objective: maximizing expected cumulative reward from the starting state. It treats the return $G_{t}$ as the total reward collected over an episode.
    - **Average reward per time step:** $J_{avg}(\theta)=\mathbb{E}_{\pi_{\theta}}(G_{t}|s)=\sum_{s}\mu(s)v^{\pi_{\theta}}(s)$. It optimizes the long-run average return and is useful in continuous tasks. But there is assumption that stationary distribution $\mu(s)$ exists.
    - **PPO(Proximal Policy Optimization)**: $$J_{\text{CLIP}}(\theta) = \mathbb{E} \left[ \min \left( r_t(\theta) A_t, \, \text{clip}(r_t(\theta), 1 - \epsilon, 1 + \epsilon) A_t \right) \right]$$
        - $ r_t(\theta) = \frac{\pi_\theta(a_t \mid s_t)}{\pi_{\theta_{\text{old}}}(a_t \mid s_t)}$
        - Regularizes the update to prevent the policy from changing too abruptly.
- 2.2 Vanilla policy gradient uses $\nabla_\theta J(\theta)=\mathbb{E}_{\pi_\theta}\left[G_t \nabla_\theta \log \pi_\theta\left(a_t \mid s_t\right)\right]$, which has high variance. But with a critic approximation $\nabla_\theta J(\theta) \approx \mathbb{E}_{\pi_\theta}\left[A^\pi\left(s_t, a_t\right) \nabla_\theta \log \pi_\theta\left(a_t \mid s_t\right)\right]$ where  $A^\pi(s, a)=Q^\pi(s, a)-V^\pi(s)$, the gradient is more stable and can lead to faster convergence with fewer samples, which improves the sample efficiency.
- 2.3 In the TRPO algorithm, we introduce a KL term in the gradient. Without the KL term, the policy might change too much in a single update, especially if the advantage estimates are noisy. The KL divergence acts like a safety brake, ensuring the new policy stays close to the old one. In other words, it limits step size in parameter space based on policy change and improves training stability.
- 2.4 This is mainly due to the followin reasons:
    - PPO is easy to implement in a few lines of code whereas TRPO requires extra codes to compute KL-divergence and perform constrained optimization.
    - PPO doesn't require second-order derivatives or matrix inversion, making it faster and lighter per training iteration.
- 2.5 The policy gradient theorem works like the following: $\nabla_{\theta}J_{\theta}(\theta) = \mathbb{E}_{\pi_{\theta}}[G_{t}\nabla_{\theta} log \pi_{\theta}(a_{t}|s_{t})]$ where $J(\theta)$ is expected return, $\pi_{\theta}(a|s)$ is policy function parametrized by $\theta$. $G_{t}$ is the return following time step $t$ and so on. $\nabla_{\theta} log \pi_{\theta}(a_{t}|s_{t})$ is the score function, telling us how policy probabilities change with parameters. Since computing this expectation would requires all possible trajectories to be ready, which is computationally intensive. Thus we need to estimate the expectation using monte carlo sampling $\nabla_\theta J(\theta) \approx \frac{1}{N} \sum_{i=1}^N G_t^{(i)} \nabla_\theta \log \pi_\theta\left(a_t^{(i)} \mid s_t^{(i)}\right)$ where $N$ is the number of sampled trajectories and $(s_{t}^{(i)}, a_{t}^{(i)}, G_{t}^{(i)})$ is sampled state, action and return at time $t$ from $i$-th trajectory.
</span>


## Exercise 3 (40 points)
### Planning with Advanced Model-based Reinforcement Learning:
- 3.1 (5 points): What are the input and output of a model of the environment in a single state Bandit problem? How about in the more general case of sequential RL problem?
- 3.2 (10 points): What’s the difference between table-lookup model, expectation model and stochastic model? Provide at least one pros and one cons for each type of model.
- 3.3 (5 points): Why is Experience Replay called a non-parametric model?
- 3.4 (5 points): In Monte-Carlo Tree Search, why does the tree policy become more optimal than the initial rollout policy during the search?
- 3.5 (15 points): Describe, in your own words, what are the differences between the AlphaGo and AlphaZero algorithms. This question is voluntarily open, and will assess your overall understanding of this technology, both in term of methodology and potential for applications

[Warning:] Responses from ChatGPT & similar lack precision relative to what we discussed
in the lecture. To get all the points, you need be significantly more precise than ChatGPT :)

<span style="color: lightgreen;" >

- 3.1 The input of the model in Bandit problem should be the current action $a$. And the output is the reward distribution of that action $p(r|a)$. The input of general case is the current state $s$ and current action $a$. The output should be the distribution over next state and reward $p(s', r|s, a)$
- 3.2 Answer
    - Table lookup Model:
        - Definition: A table-lookup model is a simple, **non-parametric** way of modeling the environment's transition and reward dynamics using a lookup table. In detail, for each state-action pair ($s, a$), the model stores:
            - Transition probabilities: $p(s'|s,a)=\frac{\# \text{times} (s, a) \rightarrow s'}{\# \text{times} (s,a)}$
            - Expected rewards: $r(s, a)=\frac{\sum_{\text{transitions from (s, a)}}r}{\# \text{times} (s,a)}$
            - These values are **learned** from empirical experience collected during interaction with the environment.
        - Pros:
            - It is very simple to implement where we just need to count occurrence and average rewards to learn the model.
            -  When the environment is small and discrete, the model gives a perfect match to empirical behavior.
        - Cons:
            - It is not ideal for large or continuous action/state spaces.
            - Every entry in the table must be visted multiple times to become accurate, which is sample inefficient in sparse or high-dimensional settings.
    - Expectation Model:
        - Definition: An expectation model is a **parametric predictive** model that linearly predicts the distribution of next state and expected reward for a given state–action pair (s,a). Typically we would have a learn function $f_\eta: s, a \longmapsto r, s^{\prime}$ such that $\quad \eta \sim \underset{\eta}{\arg \min } \mathbb{E}_{\mathbf{s}, a}\left[\left(\left(\hat{r}, \hat{\mathbf{s}}^{\prime}\right)-\left(r, \mathbf{s}^{\prime}\right)\right)^2\right]$
        - Pros:
            - Expectation models only predict a single deterministic value for each input (s,a): the mean of the next state 𝑠 and the mean reward 𝑟.
            - Training an expectation model is just supervised learning using regression where we just need $\begin{equation}\underset{\eta}{\arg \min } \mathbb{E}_{s, a}\left[\left(\left(\hat{r}, \hat{s}^{\prime}\right)-\left(r, s^{\prime}\right)\right)^2\right]\nonumber\end{equation}$
        - Cons:
            - It cannot model probability distriution as table-lookup would do since it outputs just one value(average).
            - The output point estimation may easily be biased.
    - Stochastic Model:
        - Definition: A stochastic model in reinforcement learning is a **parametric generative** model that learns the full probability distribution over possible next states and rewards 𝑟 for a given state–action pair (s,a). It is trained to approximate: $\begin{equation}g_\eta(s, a) \mapsto p(r), p\left(s^{\prime}\right)\nonumber\end{equation}$ where the model parameters $\eta$ are optimized by minimizing the negative log-likelihood: $\begin{equation}\eta \sim \arg \min _\eta \mathbb{E}_{s, a}\left[-\sum_{r, s^{\prime}} p\left(r, s^{\prime}\right) \log \hat{p}\left(r, s^{\prime}\right)\right]\nonumber\end{equation}$
        - Pros:
            - Since it is a generative model, we can utilize the output probability distribution to sample more trajectories for future use.
            - Unlike expectation model, it is more accurate and useful in stochastic environments where modeling uncertainty is essential.(e.g. autonomous driving, healthcare). 
        - Cons:
            - To learn the entire distribution over $(r, s')$, the model needs to see more examples to correctly estimate probabilities of rare or uncertain events compared with predictive model.
            - It is not ideal when fast predictions or real-time control is needed.
- 3.3 Since it is just a mechanism used in training DQN model. By storing an agent’s experiences (state, action, reward, next state) in a buffer and randomly samples them during training. This breaks temporal correlations in the data, stabilizing learning in RL. No parameters are needed to describe this procedure.
- 3.4 Since MCTS starts with a simple or random rollout policy to simulate outcomes beyond the search tree. As the tree grows and collects statistics on rewards and outcomes for different actions (e.g., using Upper Confidence Bounds or visit counts), the tree policy is updated. This causes the tree to favor more promising actions — thereby becoming more optimal over time.
- 3.5 Answer:
    - AlphaGo2016:
        - Training Pipeline: The first stage uses supervised learning to train a 13-layer SL policy network $p_{\sigma}(a|s)$ that maps the board state to a probability distribution across over all legal moves $a$ from human expert games. The second stage copies the RL policy network from first stage and use model-free policy gradient RL to improve it. The current RL policy model is trained against randomly sampled previous iteraion of policy network(self-play) for stable training. The final stage is to train a value network $v^{p}(s) \approx \mathbb{E}[z|s]$ with self-play and MSE regression to predict the game outcome from a given state where $z\in \{-1, 1\}$ is the final outcome. For every game play simulation in the second stage, it traverse the monte carlo search tree by selecting $\argmax_{a}Q(s, a) + \mu(P(s, a))$. At the end of the game play, the leaf node is evaluated with value network and rollout policy $p_{\pi}$. 
        - In short:
            - During training, the AlphaGo uses human expert data(**supervised learning**) and **self-play** to obtain SL policy network, RL policy network and value network.
            - During planning, it uses MCTS with value, **rollout policy** and policy priors to evaluate the next action.
            - It needs many CPUs for rollout evaluation.
        - Potentials for Applications: Since the training of the model requires human expert data, the algorithm is trained specifically for GO context.
    - AlphaZero2017:
        - Training Pipeline: 
           - AlphaZero only trains one deep neural network instead of three.
           - The network $f_{\theta}(**p**, v)$ takes in board state $s$ as input and outputs the probability distribution(policy) $p_{\theta}(a|s)$ of all legal actions. and a scalar $v\approx \mathbb{E}[z|s]$ like in AlphaGo2016. Thus the value estimation $v^{p}(s)$ is trained jointly with policy instead of separately.
           - The parameter $\theta$ is totally trained by self-play RL instead of combining the supervised learning(human data) and self-play.
           - During planning, it doesn't use rollout policy and only uses MCTS with value $v^{p}(s)$ and policy $p(a|s)$.
        - Potentials for Applications: Since the algorithm learns entirely via self-play(also called tabula rasa), it can be applied to different contexts like Go, Chess, Shogi, etc.
    -  In summary, AlphaGo2016 is a hybrid algorithm: starts with supervised learning, uses model-free policy gradient, and plans with MCTS + rollouts while AlphaZero2017 is a self-play algorithm: learns from scratch by repeatedly improving its policy through MCTS-guided policy iteration, and uses no rollouts or human data.
</span>